In [ ]:
import numpy
%matplotlib inline
from matplotlib import pyplot
pyplot.rc('font', family='serif', size=5)

In [ ]:
import sys
sys.path.append('../scripts/')

# Our helper
from plot_helper import *

In the previous notebook, we visually explained what are eigenvectors and eigenvalues, and we can use `numpy.linalg.eig()` to solve for the eigenpairs. Take $\,A = \begin{bmatrix} 1 & 2 \\ 2 & 1 \end{bmatrix}$ as an example.

In [ ]:
A = numpy.array([[1,2], [2,1]])
d, C = numpy.linalg.eig(A)
for eigenvalue, eigenvector in zip(d, C.T):
    print(eigenvalue, eigenvector)

$d$ is an array of eigenvalues and each column of $C$ represents a normalized eigenvector of matrix $A$. We can decompose the matrix as $A = C\, D\, C^{-1}$ where $D$ is the diagonal matrix of $d$. Let us draw both eigenvectors along with this linear transformation using `plot_linear_transformation()`, and set `unit_circle=True` to show the unit circle and `unit_vector=False` to hide the unit vectors.

In [ ]:
plot_linear_transformation(A, C[:,0], C[:,1], unit_vector=False, unit_circle=True)

The eigenvectors (in pink and dark blue) of $A$ will not change their directions after the transformation of $A$. Since $A$ is symmetric, the transformed eigenvectors happen to be the semi-axes of the transformed unit circle.

If we look at a general case where $A$ is an asymmetric matrix, say $\,A = \begin{bmatrix} 1 & 0 \\ 1 & 3 \end{bmatrix}$, we will observe that the transformed eigenvectors still keep their original directions, however, they no longer coincide with the semi-axes of the ellipse, as the plot demonstrated below. 

In [ ]:
A = numpy.array([[1,0], [1,3]])
d, C = numpy.linalg.eig(A)
plot_linear_transformation(A, C[:,0], C[:,1], unit_vector=False, unit_circle=True)

Then, how could we find the vectors that correspond to the semi-axes of the ellipse? They are intriguing because they are the most stretched or most shrunk vectors after the transformation. Similar to what we've done before, we calculate the maximum and minimum distance from the origin to the ellipse. The longest vector between them is the semi-major axis, the shortest one is the semi-minor axis.

In [ ]:
alpha = numpy.linspace(0, 2*numpy.pi, 201)
circle = numpy.vstack((numpy.cos(alpha), numpy.sin(alpha)))
ellipse = A @ circle    # 2 by 41 ndarray
distance = numpy.linalg.norm(ellipse, axis=0)
major_id = numpy.argmax(distance)
minor_id = numpy.argmin(distance)
major = ellipse[:, major_id]
minor = ellipse[:, minor_id]
print(major, numpy.linalg.norm(major))
print(minor, numpy.linalg.norm(minor))

Recall that `major` and `minor` are transformed vectors which had a unit length before the transformation. To use our plotting function `plot_linear_transformation` to visualize them, we need to pass in the corresponding `major` and `minor` vectors *before* the transformation. We name them `major_before` and `minor_before`.

In [ ]:
A_inv = numpy.linalg.inv(A)
major_before = A_inv @ major
minor_before = A_inv @ minor

We can finally plot the semi-axes now. Notice that we do not need to plot eigenvectors this time.

In [ ]:
plot_linear_transformation(A, major_before, minor_before,
                           unit_vector=False, unit_circle=True)

We all know that the major and minor axes of an ellipse are orthogonal (perpendicular to each other), however, to our surprise, their corresponding vectors `major_before` and `minor_before`, which land on the original unit circle, appear to be orthogonal as well. We can use inner product `numpy.dot()` to confirm.

In [ ]:
print(major.dot(minor))
print(major_before.dot(minor_before))

If we use $\mathbf{u_1}, \mathbf{u_2}$ to denote the *normalized* `major` and `minor` vectors, use $\mathbf{s_1}, \mathbf{s_2}$ to denote the lengths of `major` and `minor`, and use $\mathbf{v_1}, \mathbf{v_2}$ to denote `major_before` and `minor_before` (they are normalized already since they land on the unit circle), we can formulate the observation above as:

$$
\begin{align*}
  A \mathbf{v_1} = s_1 \mathbf{u_1} \\
  A \mathbf{v_2} = s_2 \mathbf{u_2}
\end{align*}
$$

Stacking two equations together:

$$
  A \begin{bmatrix}
    \mathbf{v_1} & \mathbf{v_2}
    \end{bmatrix}
    =
    \begin{bmatrix}
    \mathbf{u_1} & \mathbf{u_2}
    \end{bmatrix}
    \begin{bmatrix}
    s_1 & 0 \\
    0 & s_2
    \end{bmatrix}  
$$

Using $V$ to denote $\begin{bmatrix} \mathbf{v_1} & \mathbf{v_2} \end{bmatrix}$, $U$ to denote $\begin{bmatrix} \mathbf{u_1} & \mathbf{u_2} \end{bmatrix}$ and $S$ to denote the diagonal scaling matrix, it becomes:

$$
  A\, V = U\, S
$$

then right-multiply by $V^{-1}$ on both sides:

$$
  A = U\, S\, V^{-1}
$$

Since $\mathbf{v_1}, \mathbf{v_2}$ are orthogonal unit vectors, the matrix $V$ is an orthogonal matrix. (Recall what is an orthogonal matrix from the previous notebook) Thus, $V^T = V^{-1}$, the equation can also be written as:

$$
  A = U\, S\, V^{T}
$$

Starting from discovering the semi-axes of the transformed unit circle, we finally arrive at another neat decomposition of the matrix $A$ - singular value decomposition (SVD). The columns of $U$ are called left singular vectors, the columns of $V$ are called right singular vectors, and the diagonal elements of $S$ are singular values.

Let us build these matrices first.

In [ ]:
s1 = numpy.linalg.norm(major)
s2 = numpy.linalg.norm(minor)
S = numpy.diag([s1, s2])
print(S)

In [ ]:
u1 = major / s1
u2 = minor / s2
U = numpy.transpose(numpy.vstack((u1,u2)))
print(U)

In [ ]:
v1 = major_before
v2 = minor_before
VT = numpy.vstack((v1,v2))
print(VT)

We can retrieve matrix $A$ by multiplying them together.

In [ ]:
U @ S @ VT

From the previous notebook, we know that an orthogonal matrix ($U$ and $V$) implies a rotation transformation while a diagonal matrix ($S$) implies a scaling transformation. Let us visualize each component in the SVD. We also draw the unit vectors after each transformation this time.

In [ ]:
plot_linear_transformations(VT, S, U, unit_circle=True)

The matrix $V^T$ first "rotates" the unit circle, then the diagonal matrix $S$ "stretches" the unit circle in both $x$-axis and $y$-axis directions into an ellipse, finally the matrix $U$ "rotates" this ellipse to a new direction. Therefore, the linear transformation of matrix $A$ can be viewed as a combination of rotation and scaling.

## Compute SVD in Python

You can compute the SVD of a matrix using [`numpy.linalg.svd()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html). It returns a tuple: its first element is a 2D array where each column is a left singular vector, its second element is an array with the singular values, and its third element is a 2D array where each row is a right singular vector.

In [ ]:
U, S, VT = numpy.linalg.svd(A)

Let us loop over each pair of singular values and singular vectors, print them out.

In [ ]:
for u, s, v in zip(U.T, S, VT):
    print(s, u, v)

We can use [`numpy.allclose()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.allclose.html) to confirm that the decomposition is close enough to the matrix $A$.

In [ ]:
A_decomp = U @ numpy.diag(S) @ VT
print(A_decomp)
print(numpy.allclose(A, A_decomp))

As we discussed before, a $3 \times 3$ matrix represents a linear transformation in 3D. We can also apply SVD to it using `numpy` built-in function and visualize each component. The linear transformation is a composition of a rotation, a scaling and another rotation transformation.

In [ ]:
A = numpy.array([[1,2,3], [1,1,1], [-1,1,0]])
U, S, VT = numpy.linalg.svd(A)
plot_3d_linear_transformations(VT, numpy.diag(S), U, unit_sphere=True)

### Rank-deficient matrix

In the second notebook, we introduced the rank of a matrix, the dimension of the vector space spanned by the columns of a matrix. It also describes the number of linearly independent columns of a matrix. When the rank of a matrix does not match the number of columns, the matrix is called *rank-deficient*.

The matrix $N = \begin{bmatrix} 1 & 2 & 7 \\
                                0 & 1 & 3 \\
                                -3 & 1 & 0 \end{bmatrix} $ has rank 2. Plotting this information below, you will find that all the 3d grid lines collapse on a 2d plane.

In [ ]:
N = numpy.array([[1,2,7], [0,1,3], [-3,1,0]])
plot_3d_linear_transformation(N)

With the help of SVD, now we could look at this rank-deficient matrix from a different perspective. Let's plot each component of SVD and turn on `unit_sphere`.

In [ ]:
U, S, VT = numpy.linalg.svd(N)
plot_3d_linear_transformations(VT, numpy.diag(S), U, unit_sphere=True)

The second transformation squishes the unit sphere into an ellipse on the plane $z=0$. Since each singular value describes the scaling ratio along the corresponding axis, what we observed implies that the third singular value, corresponding to $z$-axis, is zero. Let's print out the singular values to verify.

In [ ]:
print(S)

For a $3 \times 3$ matrix, the number of nonzero singular values determines the dimensionality of the unit sphere after the transformation. If the matrix is full-rank, the transformed shape is an ellipsoid; if it has rank 2, the transformed shape is an ellipse; if it has rank 1, the transformed shape is a line segment. We can generalize this property to higher dimensions as follows:

##### Key idea:

> The number of nonzero singular values of square matrix $A$ equals to its rank.

##### Challenge:

> Check the singular values of a rank-1 matrix.

For a $n \times n$ matrix, SVD can be written as:

$$
  A 
    =
    \begin{bmatrix}
    \mathbf{u_1} & \mathbf{u_2} & \dots & \mathbf{u_n}
    \end{bmatrix}
    \begin{bmatrix}
    s_1 & 0 & \dots & 0 \\
    0 & s_2 & \dots & 0 \\
    \vdots & \vdots & & \vdots \\
    0 & 0 & \dots & s_n
    \end{bmatrix}   
    \begin{bmatrix}
    \mathbf{v_1}^T \\
    \mathbf{v_2}^T \\
    \vdots \\
    \mathbf{v_n}^T
    \end{bmatrix}
$$

When a matrix is rank-deficient with rank $k$, only $k$ singular values are non-zero, the decomposition can be truncated as:

$$
  A 
    =
    \begin{bmatrix}
    \mathbf{u_1} & \mathbf{u_2} & \dots & \mathbf{u_k}
    \end{bmatrix}
    \begin{bmatrix}
    s_1 & 0 & \dots & 0 \\
    0 & s_2 & \dots & 0 \\
    \vdots & \vdots & & \vdots \\
    0 & 0 & \dots & s_k
    \end{bmatrix}   
    \begin{bmatrix}
    \mathbf{v_1}^T \\
    \mathbf{v_2}^T \\
    \vdots \\
    \mathbf{v_k}^T
    \end{bmatrix}
$$

Only the first $k$ singular vectors are needed because the singular values of the rest singular vectors are zero. In a geometrical view, if a vector's component along an axis is squished to zero, it does not matter how you rotate that component to other axes.

In the example above, the $3 \times 3$ matrix $N$ has rank 2.

In [ ]:
k = numpy.linalg.matrix_rank(N)
print(k)

We can reconstruct $N$ with only the first two singular vectors and singular values:

In [ ]:
U, S, VT = numpy.linalg.svd(N)
N_k = U[:,:k] @ numpy.diag(S[:k]) @ VT[:k,:]
print(N_k)
print(numpy.allclose(N, N_k))

This implies a great advantage of low-rank matrices: they can be stored and reconstructed inexpensively by stripping zero singular values and the corresponding singular vectors from SVD.

### Low Rank Approximation

Though low-rank matrices are easier to work with, the matrices from real-world applications often are not in our favor. Take a look at the $4 \times 4$ matrix
$ A = \begin{bmatrix} 1 & 2 & 3 & 6 \\
                      2 & 5 & 7 & 10 \\
                      3 & 9 & 12 & 14 \\
                      4 & 7 & 9 & 15 \end{bmatrix} $.
Let's first check the rank of $A$.

In [ ]:
A = numpy.array([[1, 2, 3, 6],
                 [2, 5, 7, 10],
                 [3, 9, 12, 14],
                 [4, 7, 9, 15]])
print(numpy.linalg.matrix_rank(A))

It is difficult to visualize this linear transformation with a plot, but we can gain some insight by doing SVD. All four singular values are non-zero, however, the last one is significantly smaller than others and close to zero, indicating that the "4d unit sphere" is almost stamped into a "plane" in that direction. In other words, it has full rank but is very close rank-deficient (with rank $3$).

In [ ]:
U, S, VT = numpy.linalg.svd(A)
print(S)

Why not try to construct the matrix $A$ with the three largest singular values?

In [ ]:
k = 3
A_approx = U[:,:k] @ numpy.diag(S[:k]) @ VT[:k,:]
print(A_approx)

By dropping smaller singular values and their singular vectors in the decomposition, we arrive at a rank-$3$ approximation of matrix $A$.

### Image Compression

In [ ]:
from imageio import imread, imwrite
image = imread('../images/washington-monument.jpg')

In [ ]:
print(type(image))
print(image.shape)
print(image.dtype)

In [ ]:
pyplot.figure(figsize=(2,2))
pyplot.imshow(image, cmap='gray');

In [ ]:
U, S, VT = numpy.linalg.svd(image)
print(U.shape, S.shape, VT.shape)

In [ ]:
pyplot.figure(figsize=(2,2))
pyplot.scatter(numpy.arange(len(S)), S, s=0.8)
pyplot.yscale('log')
pyplot.ylabel('singular value');

In [ ]:
from ipywidgets import widgets, fixed

In [ ]:
def approximate(k, u, s, vt, image):
    u_k = u[:,:k]
    s_k = s[:k]
    vt_k = vt[:k,:]
    copy = u_k @ numpy.diag(s_k) @ vt_k
    diff = numpy.abs(image - copy)
    fig = pyplot.figure(figsize=(4,2))
    ax1 = pyplot.subplot(121)
    ax1.imshow(copy, cmap='gray')
    ax1.set_title('compressed image'.format(k))
    ax2 = pyplot.subplot(122)
    ax2.imshow(image, cmap='gray')
    ax2.set_title('original image'.format(k))
    
    ratio = image.size / (u_k.size + s_k.size + vt_k.size)
    fig.suptitle('$k = {}$, compression ratio = {:.2f}'.format(k, ratio))

In [ ]:
slider = widgets.IntSlider(min=1, max=101, step=5)
widgets.interact(approximate, k=slider, u=fixed(U),
                 s=fixed(S), vt=fixed(VT), image=fixed(image));

### Nonsquare matrix

So far, we have only talked about square matrices. A $2\times 2$ matrix transforms a 2D vector to another 2D vector, and a $3\times 3$ matrix transforms a 3D vector to another 3D vector. What about the nonsquare ones, for example, a $3 \times 2$ matrix $A =  \begin{bmatrix} 1 & 0 \\ 1 & 1 \\ 2 & 3\end{bmatrix}$ ?

In the first lesson, we discussed that the matrix-vector multiplication $A\mathbf{x}$ is a linear combination of the columns of $A$ scaled by the components of $\mathbf{x}$. Since $A$ has two columns, $\mathbf{x}$ should have two components. Each column of $A$ is a 3D vector, therefore, the linear combination of the columns $A\mathbf{x}$ should also be a 3D vector. Yes, this matrix $A$ transforms a 2D vector into a 3D vector.

If we use $\mathbf{i}$ and $\mathbf{j}$ to denote the standard basis vectors. The first column corresponds to the vector where $\mathbf{i}$ lands after the transformation, and the second column is where $\mathbf{j}$ lands:
$$
\mathbf{i} = \begin{bmatrix} 1 \\ 0 \end{bmatrix}  \Rightarrow  \begin{bmatrix} 1 \\ 1 \\ 2 \end{bmatrix} \\
\mathbf{j} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}  \Rightarrow  \begin{bmatrix} 0 \\ 1 \\ 3 \end{bmatrix}
$$

so a vector $\mathbf{x} = \begin{bmatrix} x \\ y \end{bmatrix}$ will land in $ A\mathbf{x} = x \cdot \begin{bmatrix} 1 \\ 1 \\ 2 \end{bmatrix} + y \cdot \begin{bmatrix} 0 \\ 1 \\ 3 \end{bmatrix}$.

##### Key idea:

> An $m\times n$ nonsquare matrice transforms an $n$-dimensional vector to an $m$-dimensional vector.

Matrix-matrix multiplication represents the composition of two linear transformations. For nonsquare matrices, we need to make sure that the dimension of the transformed vector of the previous transformation matches the dimension of the input vector of the next transformation. If matrix $A$ is $m\times n$, matrix $B$ is $n\times p$, the matrix product $AB$ is $m \times p$.

### SVD in general

In fact, SVD also exists for nonsquare matrices. An $m \times n$ matrix $M$ can be factorized as:

$$
\underset{m\times n}M = \underset{m\times m}U\,\, \underset{m\times n}S\,\,
\underset{n\times n}{V^T}
$$

$U$ and $V$ are orthogonal matrices, and they can treated as rotation transformations. $S$ is an $m \times n$ rectangular diagonal matrix and still represents a scaling transformation, but in contrast to square matrices, $S$ now transforms an $n$-dimensional vector to an $m$-dimensional vector.

We can use the same built in function `numpy.linalg.svd()` to decompose rectangular matrices. Take the following $3\times 4$ matrix as an example:

In [ ]:
M = numpy.array([[1, 3, 4, 4],
                 [2, 5, 7, 10],
                 [3, 4, -1, 0]], dtype=float)
u, s, vt = numpy.linalg.svd(M)

Let's check the shape of each component.

In [ ]:
print(u.shape, s.shape, vt.shape)

The second return value `s` is a vector, it only keeps the diagonal elements of $S$. In the example above, we used `numpy.diag(s)` to construct $S$ of that square matrix. The same code won't work here because $S$ has to be a $3\times 4$ matrix in this case, while `numpy.diag(s)` only creates a $3 \times 3$ matrix. 

In [ ]:
u @ numpy.diag(s) @ vt

To fix it, first we need to create $S$ with the correct shape using `numpy.zeros_like()`, then fill the diagonal with the singular values.

In [ ]:
S = numpy.zeros_like(M, dtype=float)
S[:len(s), :len(s)] = numpy.diag(s)

We can now reconstruct the rectangular matrix $M$ using the three components.

In [ ]:
u @ S @ vt

### Pseudo-inverse

From the previous notebook, we treat a matrix's inverse as the reverse linear transformation that brings the transformed vectors back to their original coordinates. So far, we have only considered square matrices, and only the ones with full rank have an inverse. What about the inverse of a rectangular matrix? Have a try using the built-in function `numpy.linalg.inv()`.


The inverse of a nonsquare matrix does not exist. You can think of a nonsquare matrix as the coefficient matrix of a linear system of equations. Since the number of equations is not equal to the number of unknowns, there is no unique solution to the system, thus we cannot find the inverse of a nonsquare matrix.

Despite lack of an inverse, nonsquare matrices do have a generalized form of inverse, called *pseudo-inverse*. It satisfies weaker conditions than the standard inverse and exists for all rectangular matrices. The idea of this generalized inverse is closed related to SVD. Given the SVD of an $m \times n$ matrix $M = U \, S\, V^T$, the pseudo-inverse $M^+$ is defined as:

$$
\underset{n\times m}M^+ = \underset{n\times n}V \,\, \underset{n\times m}{S^+} \,\, \underset{m\times m}{U^T}
$$

where $S^+$ comes from $S$ by taking the reciprocal of the nonzero diagonal elements of $S$, all other elements in $S^+$ are zero. If the diagonal elements of $S$ are $(s_1, s_2, \dots, s_r, 0, \dots, 0)$, the diagonal elements of $S^+$ are $(\frac{1}{s_1}, \frac{1}{s_2}, \dots, \frac{1}{s_r}, 0, \dots, 0)$, where $r$ denotes the number of nonzero singular values.

In [ ]:
M_inv @ M @ M_inv

In [ ]:
M_inv = numpy.linalg.pinv(M)

In [ ]:
M @ M_inv

In [ ]:
M_inv

### Linear Regression with SVD

In [ ]:
from urllib.request import urlretrieve
URL = 'http://go.gwu.edu/engcomp1data5?accessType=DOWNLOAD'
fname = 'land_global_temperature_anomaly-1880-2016.csv'
urlretrieve(URL, fname)

In [ ]:
year, temp_anomaly = numpy.loadtxt(fname, delimiter=',', skiprows=5, unpack=True)

In [ ]:
#You can set the size of the figure by doing:
pyplot.figure(figsize=(3,2))

#Plotting
pyplot.plot(year, temp_anomaly, color='#2929a3', linestyle='-', linewidth=0.8) 
pyplot.title('Land global temperature anomalies.')
pyplot.xlabel('Year')
pyplot.ylabel('Land temperature anomaly [°C]')
pyplot.grid();

In [ ]:
# from sklearn.linear_model import LinearRegression
# reg = LinearRegression().fit(year.reshape(-1,1), temp_anomaly)
# #You can set the size of the figure by doing:
# pyplot.figure(figsize=(3,2))

# #Plotting
# pyplot.plot(year, temp_anomaly, color='#2929a3', linestyle='-', linewidth=0.8)
# pyplot.plot(year, reg.predict(year.reshape(-1,1)))
# pyplot.title('Land global temperature anomalies.')
# pyplot.xlabel('Year')
# pyplot.ylabel('Land temperature anomaly [°C]')
# pyplot.grid();

In [ ]:
A = numpy.zeros((137,2))
A[:,0] = 1
A[:,1] = year

In [ ]:
U, S, VT = numpy.linalg.svd(A)
print(U.shape, S.shape, VT.shape)

In [ ]:
S_inv = numpy.zeros((2, 137))
S_inv[:2, :2] = numpy.diag(1/S)

In [ ]:
A_inv = VT.T @ S_inv @ U.T

In [ ]:
b, k = A_inv @ temp_anomaly

In [ ]:
#You can set the size of the figure by doing:
pyplot.figure(figsize=(3,2))

#Plotting
pyplot.plot(year, temp_anomaly, color='#2929a3', linestyle='-', linewidth=0.5)
#pyplot.plot(year, reg.predict(year.reshape(-1,1)), linestyle='-.', linewidth=0.5)
pyplot.plot(year, b + k*year, linestyle='--', linewidth=0.5)
pyplot.title('Land global temperature anomalies.')
pyplot.xlabel('Year')
pyplot.ylabel('Land temperature anomaly [°C]')
pyplot.grid();

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())